# 합성곱

밀집층(fc) 신경망에서는, 입력의 수에 상관없이 뉴런의 수가 출력의 수를 결정했다.

합성곱(conv) 신경망에서는, 가중치의 수가 필터(커널)에 의해 정해지고 (하나의 뉴런이 정해지고), 그 필터를 입력 데이터의 순서대로 덮어가며 출력을 계산한다.

합성곱에서는 뉴런이 입력 위를 이동하면서 출력을 만들기 때문에, fc와 같은 형태로 그림을 표현하기가 어렵다.
- 뉴런이라고 부르기도 어색하다.
- 합성곱 신경망(CNN)에서는 FC 신경망과는 달리 뉴런은 필터(커널)라고 부른다.
    - 커널: 입력에 곱하는 가중치를 표현할 때
    - 필터: 뉴런 개수를 표현할 때

특성 맵(feature map): 합성곱을 통해 얻은 출력을 특별히 가리키는 말

밀집층에서 여러 개의 뉴런을 사용하듯, 합성곱 층에서도 여러 개의 필터를 사용할 수 있다.
- 여러 개의 특성 맵을 차곡차곡 쌓아, 한 차원 더 확장된 특성 맵을 얻을 수 있다.
- 이 때 필터는 당연히 가중치가 서로 다르다.

2차원 구조를 그대로 사용할 수 있기 때문에, 합성곱 신경망이 이미지 처리 분야에서 뛰어난 성능을 발휘한다.

# 케라스 합성곱 층

합성곱 신경망(convolution neural network)
- 일반적으로 1개 이상의 Conv 층을 쓴 인공 신경망을 CNN이라고 부른다.
- 분류 문제 등 보통 맨 마지막 충은 FC로 두는 것이 일반적이기 때문에, conv 층으로만 이루어진 것이 CNN인 것은 아니다.

케라스에서는 keras.layers.Conv2D 클래스를 제공한다.
- 첫 번째 매개변수: 필터의 개수. (필수)
- kernal_size: 필터로 사용할 커널의 크기. (필수)
- activation: 활성화 함수
    - conv를 거칠 때 활성화 함수를 통과한다. --> 특성 맵은 활성화 함수 이후의 값

In [1]:
from tensorflow import keras

keras.layers.Conv2D(10, kernel_size=(3, 3), activation='relu')

패딩(padding): 입력 배열의 주위를 가상의 원소로 채우는 것
- same padding: 입력과 특성 맵의 크기를 동일하게 만들기 위해, 입력 주위를 0으로 패딩하는 것
    - Conv2D()의 매개변수 padding='same'
- valid padding: 패딩 없이 순수한 입력 배열에서만 합성곱을 하여 특성 맵을 만드는 경우. 특성 맵의 크기가 줄어들 수 밖에 없다.
    - Conv2D()의 매개변수 padding='valid' (default)

패딩을 통해 얻는 이점
- 가운데 있는 원소와 모서리에 있는 원소의 정보를 좀 더 동등하게 특성 맵으로 전달할 수 있다.

In [2]:
keras.layers.Conv2D(10, kernel_size=(3, 3), activation='relu', padding='same')

스트라이드(stride): 커널의 이동 간격 (default: 1)

In [3]:
keras.layers.Conv2D(10, kernel_size=(3, 3), activation='relu', padding='same', strides=1)

풀링(pooling): 특성 맵의 크기를 줄이는 역할
- 단, 특성 맵의 개수 자체는 줄이지 않는다.
- 필터와 마찬가지로 입력 위를 지나가면서 도장을 찍는다.
    - 대신 가중치가 없다.
    - 영역에서 가장 큰 값을 고르거나(max pooling), 평균값을 계산한다(average pooling).
- 풀링은 겹치는 공간 없이 입력 위를 이동한다.

keras.layers.MaxPooling2D() 혹은 .AveragePooling2D()
- 첫 번째 매개변수: 풀링의 크기
    - 가로 세로 크기를 달리하려면 정수의 튜플로 지정할 수 있다.
- Conv2D와 마찬가지로 strides와 padding 매개변수를 제공하지만, 사실 쓸 일이 없다.
- 보통 평균 풀링보다 최대 풀링을 더 많이 사용한다.
    - 평균 풀링은 중요한 정보를 희석시킬 수 있기 때문에

In [4]:
keras.layers.MaxPooling2D(2)
# == keras.layers.MaxPooling2D(2, strides=2, padding='valid')

# 합성곱 신경망의 전체 구조

입력 (4, 4) + 한줄 패딩(same padding)

\+ (3, 3) 필터 3개

--> (4, 4, 3) 특성 맵 (활성화 함수 적용)

\+ (2, 2) 최대 풀링

--> (2, 2, 3) 특성 맵

\+ Flatten

--> (12, ) 크기 입력

\+ FC + softmax --> 예측

컬러 이미즈를 사용한 Conv
- RGB 이미지를 각 채널로 분리하여 3차원 배열로 나타낸다.
- 그렇기 때문에 커널도 3차원으로 확장하여야 함.

사실 케라스의 CNN은 항상 3차원 입력을 기대한다.
- 따라서, 패션 MNIST 데이터처럼 흑백 이미지인 경우, 깊이가 1인 3차원 배열로 변환하여 전달한다.
    - (28, 28) --> (28, 28, 1)

그 외에도, 여러 필터를 거쳐 만들어낸 3차원 특성 맵을 다시 Conv 층에 넣는 경우도 3차원 입력의 예이다.

아무튼 마지막에 FC를 넣기 전엔 Flatten 해준다.